# GPT-2 for music - By Dr. Tristan Behrens

This notebook shows you how to generate music with GPT-2

---

## Find me online

- https://www.linkedin.com/in/dr-tristan-behrens-734967a2/
- https://twitter.com/DrTBehrens
- https://github.com/AI-Guru
- https://huggingface.co/TristanBehrens
- https://huggingface.co/ai-guru


---

## Install depencencies.

The following cell sets up fluidsynth and pyfluidsynth on colaboratory.

In [31]:
if "google.colab" in str(get_ipython()):
    print("Installing dependencies...")
    #!pip uninstall -y bokeh
    !apt-get update -qq && apt-get install -qq  build-essential libasound2-dev libjack-dev && apt-get install libfluidsynth3
    !pip install -qU pyfluidsynth
    #!pip install --upgrade bokeh==2.4.3

Installing dependencies...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libfluidsynth3 is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [32]:
!pip install transformers
!pip install note_seq

## Load the tokenizer and the model from 🤗 Hub.

In [33]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [34]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-guru/lakhclean_mmmtrack_4bars_d-2048")
model = AutoModelForCausalLM.from_pretrained("ai-guru/lakhclean_mmmtrack_4bars_d-2048")

## Convert the generated tokens to music that you can listen to.

This uses note_seq, which is something like MIDI coming from Google Magenta. You could even use it to load and save MIDI files. Check their repo if you want to learn more.


In [35]:
import note_seq

NOTE_LENGTH_16TH_120BPM = 0.25 * 60 / 120
BAR_LENGTH_120BPM = 4.0 * 60 / 120

def token_sequence_to_note_sequence(token_sequence, use_program=True, use_drums=True, instrument_mapper=None, only_piano=False):

    if isinstance(token_sequence, str):
        token_sequence = token_sequence.split()

    note_sequence = empty_note_sequence()

    # Render all notes.
    current_program = 1
    current_is_drum = False
    current_instrument = 0
    track_count = 0
    for token_index, token in enumerate(token_sequence):

        if token == "PIECE_START":
            pass
        elif token == "PIECE_END":
            print("The end.")
            break
        elif token == "TRACK_START":
            current_bar_index = 0
            track_count += 1
            pass
        elif token == "TRACK_END":
            pass
        elif token == "KEYS_START":
            pass
        elif token == "KEYS_END":
            pass
        elif token.startswith("KEY="):
            pass
        elif token.startswith("INST"):
            instrument = token.split("=")[-1]
            if instrument != "DRUMS" and use_program:
                if instrument_mapper is not None:
                    if instrument in instrument_mapper:
                        instrument = instrument_mapper[instrument]
                current_program = int(instrument)
                current_instrument = track_count
                current_is_drum = False
            if instrument == "DRUMS" and use_drums:
                current_instrument = 0
                current_program = 0
                current_is_drum = True
        elif token == "BAR_START":
            current_time = current_bar_index * BAR_LENGTH_120BPM
            current_notes = {}
        elif token == "BAR_END":
            current_bar_index += 1
            pass
        elif token.startswith("NOTE_ON"):
            pitch = int(token.split("=")[-1])
            note = note_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + 4 * NOTE_LENGTH_16TH_120BPM
            note.pitch = pitch
            note.instrument = current_instrument
            note.program = current_program
            note.velocity = 80
            note.is_drum = current_is_drum
            current_notes[pitch] = note
        elif token.startswith("NOTE_OFF"):
            pitch = int(token.split("=")[-1])
            if pitch in current_notes:
                note = current_notes[pitch]
                note.end_time = current_time
        elif token.startswith("TIME_DELTA"):
            delta = float(token.split("=")[-1]) * NOTE_LENGTH_16TH_120BPM
            current_time += delta
        elif token.startswith("DENSITY="):
            pass
        elif token == "[PAD]":
            pass
        else:
            #print(f"Ignored token {token}.")
            pass

    # Make the instruments right.
    instruments_drums = []
    for note in note_sequence.notes:
        pair = [note.program, note.is_drum]
        if pair not in instruments_drums:
            instruments_drums += [pair]
        note.instrument = instruments_drums.index(pair)

    if only_piano:
        for note in note_sequence.notes:
            if not note.is_drum:
                note.instrument = 0
                note.program = 0

    return note_sequence

def empty_note_sequence(qpm=120.0, total_time=0.0):
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = qpm
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = total_time
    return note_sequence

## Generate music

This will generate one track of music and render it.

In [36]:
generated_sequence = "PIECE_START"



Note: Run the following cell multiple times to generate more tracks.

In [41]:


# Encode the conditioning tokens.
input_ids = tokenizer.encode(generated_sequence, return_tensors="pt")
#print(input_ids)

# Generate more tokens.
eos_token_id = tokenizer.encode("TRACK_END")[0]
temperature = 1.0
generated_ids = model.generate(
    input_ids,
    max_length=2048,
    do_sample=True,
    temperature=temperature,
    eos_token_id=eos_token_id,
)
generated_sequence = tokenizer.decode(generated_ids[0])
print(generated_sequence)

note_sequence = token_sequence_to_note_sequence(generated_sequence)

synth = note_seq.fluidsynth
note_seq.plot_sequence(note_sequence)
note_seq.play_sequence(note_sequence, synth)

PIECE_START TRACK_START INST=3 DENSITY=9 BAR_START NOTE_ON=38 TIME_DELTA=2 NOTE_ON=69 NOTE_ON=76 NOTE_ON=74 NOTE_ON=78 TIME_DELTA=1 NOTE_OFF=38 NOTE_OFF=69 NOTE_OFF=76 NOTE_OFF=74 NOTE_OFF=78 TIME_DELTA=1 NOTE_ON=40 NOTE_ON=78 NOTE_ON=74 NOTE_ON=69 TIME_DELTA=2 NOTE_OFF=40 NOTE_OFF=78 NOTE_OFF=74 NOTE_OFF=69 TIME_DELTA=2 NOTE_ON=38 NOTE_ON=38 TIME_DELTA=3 NOTE_OFF=38 NOTE_OFF=38 TIME_DELTA=1 NOTE_ON=38 NOTE_ON=38 TIME_DELTA=3 NOTE_OFF=38 NOTE_OFF=38 BAR_END BAR_START NOTE_ON=43 NOTE_ON=43 TIME_DELTA=2 NOTE_OFF=43 NOTE_OFF=43 NOTE_ON=43 NOTE_ON=43 TIME_DELTA=2 NOTE_OFF=43 NOTE_OFF=43 TIME_DELTA=2 NOTE_ON=38 NOTE_ON=41 TIME_DELTA=1 NOTE_OFF=38 NOTE_OFF=41 TIME_DELTA=1 NOTE_ON=40 NOTE_ON=40 TIME_DELTA=1 NOTE_OFF=40 NOTE_OFF=40 TIME_DELTA=1 NOTE_ON=38 NOTE_ON=38 TIME_DELTA=1 NOTE_OFF=38 NOTE_OFF=38 TIME_DELTA=1 NOTE_ON=39 NOTE_ON=39 TIME_DELTA=2 NOTE_OFF=39 NOTE_OFF=39 NOTE_ON=40 NOTE_ON=40 TIME_DELTA=1 NOTE_OFF=40 NOTE_OFF=40 BAR_END BAR_START NOTE_ON=95 NOTE_ON=95 NOTE_ON=89 TIME_DELTA=1

# Thank you!